In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
import pandas as pd

df =  pd.read_csv('.//datasets//spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

we can see our data is imbalanced

In [5]:
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [6]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
from sklearn.model_selection import train_test_split as tts

In [10]:
x_train, x_test, y_train, y_test = tts(df['Message'], df['spam'], test_size=0.2,
                                      stratify=df['spam'])

In [11]:
y_train.value_counts()

spam
0    3859
1     598
Name: count, dtype: int64

# loading BERT model

In [12]:
preprocess_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-l-12-h-768-a-12/4"

In [13]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

bert_model = hub.KerasLayer(encoder_url)

In [15]:
def get_sentence_embeding(sentence):
    preprocessed_text = bert_preprocess_model(sentence)
    return bert_model(preprocessed_text)['pooled_output']

In [16]:
v = get_sentence_embeding([
    'banana',
    'grapes',
    'mango',
    'jeff bezos',
    'elon musk',
    'bill gates'
])

now we can compare vectors with cosine similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([v[0]],[v[1]])  # banana and grapes are very similar

array([[0.99110895]], dtype=float32)

In [22]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocess_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocess_text)

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

model = tf.keras.Model(inputs=[text_input], outputs= [l])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [23]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=2)

Epoch 1/2
 56/140 [===========>..................] - ETA: 10:17 - loss: 0.3749 - accuracy: 0.8583

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)